In [2]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare

url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [5]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 10.9 s
Wall time: 3min 59s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 31.8 s
Wall time: 32.3 s


In [30]:
os.chdir("figsharerainfall")

In [31]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd

# files = glob.glob('figsharerainfall/*.csv')
extension = 'csv'
all_filenames = glob.glob('*.{}'.format(extension))
df = pd.concat((pd.read_csv(file, index_col=0)
                # .assign(model=re.findall(r'(?<=\/)[^_]+(?=_)', file))
                .assign(model=re.findall(r'^[^_]+(?=_)', file)[0])
                for file in all_filenames)
              )

# files = glob.glob('figsharerainfall/*.csv')
# df = pd.concat((pd.read_csv(file, index_col=0)
#                 .assign(model = file.strip('figsharerainfall\\').split('_')[0])
#                 for file in files))
df.to_csv("combined_data.csv")

CPU times: total: 6min 34s
Wall time: 6min 37s


# Simple EDA

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Member 1    |                  |     |           |        |            |
| Member 2    |                  |     |           |        |            |
| Member 3    |                  |     |           |        |            |
| Member 4    |                  |     |           |        |            |

In [32]:
df.tail()


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
2014-12-27 12:00:00,-30.157068,-29.21466,153.125,154.375,0.554375,TaiESM1
2014-12-28 12:00:00,-30.157068,-29.21466,153.125,154.375,7.028577,TaiESM1
2014-12-29 12:00:00,-30.157068,-29.21466,153.125,154.375,0.234757,TaiESM1
2014-12-30 12:00:00,-30.157068,-29.21466,153.125,154.375,2.097459,TaiESM1
2014-12-31 12:00:00,-30.157068,-29.21466,153.125,154.375,0.548421,TaiESM1


In [33]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [34]:
# Checking the dtype of each column
df.info()
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")

<class 'pandas.core.frame.DataFrame'>
Index: 62513863 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float64
 1   lat_max        float64
 2   lon_min        float64
 3   lon_max        float64
 4   rain (mm/day)  float64
 5   model          object 
dtypes: float64(5), object(1)
memory usage: 3.3+ GB
Memory usage with float64: 3500.78 MB


# Changing the dtype

In [36]:
# Changing the dtype of numerical columns from float64 to float32 
column_name = list(df.select_dtypes(include=['float64']))
for name in column_name:
    df[name] = df[name].astype('float32')

In [37]:
df.info()
print(f"Memory usage with float32: {df.memory_usage().sum() / 1e6:.2f} MB")

<class 'pandas.core.frame.DataFrame'>
Index: 62513863 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float32
 1   lat_max        float32
 2   lon_min        float32
 3   lon_max        float32
 4   rain (mm/day)  float32
 5   model          object 
dtypes: float32(5), object(1)
memory usage: 2.1+ GB
Memory usage with float32: 2250.50 MB


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Member 1    |                  |     |           |        |            |
| Member 2    |                  |     |           |        |            |
| Member 3    |                  |     |           |        |            |
| Member 4    |                  |     |           |        |            |

Check if time can be changed to datetime object. 

In [ ]:
# df.reset_index()
# # df['dates'] = pd.to_datetime(df['dates'], format='%Y%m%d')

In [38]:
%%time
df1 = pd.read_csv("combined_data.csv")
print(df1["model"].value_counts())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
MRI-ESM2-0          3037320
EC-Earth3-Veg-LR    3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
NorESM2-LM           919800
CanESM5              551880
BCC-ESM1             551880
observed              46020
Name: model, dtype: int64
CPU times: total: 59.4 s
Wall time: 1min 1s


In [39]:
df1.shape

(62513863, 7)

## Loading it by Chunks

In [40]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
AWI-ESM-1-1-LR       966420
BCC-CSM2-MR         3035340
BCC-ESM1             551880
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
CanESM5              551880
EC-Earth3-Veg-LR    3037320
FGOALS-f3-L         3219300
FGOALS-g3           1287720
GFDL-CM4            3219300
GFDL-ESM4           3219300
INM-CM4-8           1609650
INM-CM5-0           1609650
KIOST-ESM           1287720
MIROC6              2070900
MPI-ESM-1-2-HAM      966420
MPI-ESM1-2-HR       5154240
MPI-ESM1-2-LR        966420
MRI-ESM2-0          3037320
NESM3                966420
NorESM2-LM           919800
NorESM2-MM          3541230
SAM0-UNICON         3541153
TaiESM1             3541230
observed              46020
dtype: int32
CPU times: total: 51 s
Wall time: 51 s


In [42]:
%%time
print(df['rain (mm/day)'].min())

-3.807373e-12
CPU times: total: 1.08 s
Wall time: 1.09 s


In [41]:
%%time
df.describe()

CPU times: total: 8.77 s
Wall time: 9.38 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.929456e+07
mean,-3.310463e+01,-3.197746e+01,1.469054e+02,1.482152e+02,1.901829e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.588274e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.876672e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.161705e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.021314e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [43]:
%%time
df.isnull().sum()

CPU times: total: 4.75 s
Wall time: 4.75 s


lat_min          3265320
lat_max            46020
lon_min          3265320
lon_max            46020
rain (mm/day)    3219300
model                  0
dtype: int64

## Loading only those columns that are needed

In [44]:
%%time
use_cols = ['rain (mm/day)', 'model']
df3 = pd.read_csv("combined_data.csv", usecols = use_cols)
df3.groupby('model').mean()


CPU times: total: 32.5 s
Wall time: 32.6 s


,rain (mm/day),model
0,3.293256e-13,ACCESS-CM2
1,0.000000e+00,ACCESS-CM2
2,0.000000e+00,ACCESS-CM2
3,0.000000e+00,ACCESS-CM2
4,1.047658e-02,ACCESS-CM2


## Loading only those columns that are needed

In [51]:
%%time
# use_cols = ['rain (mm/day)', 'lat_min','lon_min','time']
df4 = pd.read_csv("combined_data.csv", index_col = 0, parse_dates = True)
df4.head()

CPU times: total: 1min 18s
Wall time: 1min 41s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [ ]:
df5 = df4[(df4.lat_min == -36.25) & (df4.lon_min == 140.625) & model = "ACCESS-CM2"]['']
# print(df5.value_counts())
# df5.plot.line(xlabel="time", ylabel="rain (mm/day)", legend=False);

In [ ]:
%%time
mean_rain = df3.groupby('model').mean()

bar = (alt.Chart(df3).mark_bar().encode(
    alt.X(mean_rain, type='quantitative', scale=alt.Scale(zero=False)),
    y = 'model'))
    
bar